In [1]:
import sys
import os

# Add the parent directory of 'vegetation-period-NDVI' to the sys.path
sys.path.append(os.path.abspath(os.path.join('..')))


In [2]:
import ee
import geemap
from time_series import extract_time_ranges, get_harmonic_ts
from vegetation_period_extraction import (
    get_crop_veg_period,
    create_binary_ndvi_indicator,
    create_binary_mask,
)
from typing import List

In [ ]:
ee.Initialize(project="thurgau-irrigation")

## Exporting the vegetation period images as an asset to the project

### Define the Area of Interest

In [ ]:
cantonal_borders_asset = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"

aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
aoi_geometry = aoi_feature_collection.geometry()
aoi_geometry = aoi_geometry.simplify(500)
aoi_buffered = aoi_geometry.buffer(100)

aoi_name = "Thurgau"

### Define the years of interest

In [ ]:
years = [2019]

### Export the Vegetation Periods images to the project assets

In [ ]:
for year in years:

    print(f"Processing year {year}")

    time_intervals = extract_time_ranges([f"{year}-03-01", f"{year}-10-31"], 15)

    image_to_export = get_crop_veg_period(year, aoi_buffered, time_intervals)

    task_name = f"Thurgau_crop_veg_period_{year}"
    asset_id = f"projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_{year}"

    bands_to_export = ["firstStart", "firstEnd", "secondStart", "secondEnd", "isDoubleCropping"]

    image_to_export = image_to_export.select(bands_to_export)


    task = ee.batch.Export.image.toAsset(
        image=image_to_export,
        description=task_name,
        assetId=asset_id,
        region=aoi_buffered,
        scale=10,
        maxPixels=1e13,
    )

    task.start()

    print(f"Exporting {task_name} to {asset_id}")

## General Code Testing. Ignore this part

In [ ]:
year = 2019

# Define the time range for analysis
start_date = f"{year}-03-01"
end_date = f"{year}-10-31"

# Create time intervals
time_intervals = extract_time_ranges([start_date, end_date], 15)

# Get the vegetation period estimation
veg_period_image = get_crop_veg_period(year, aoi_buffered, time_intervals)

In [ ]:
# type(veg_period_image)

In [ ]:
double_band = veg_period_image.select('isDoubleCropping')

# Create a map
Map = geemap.Map(zoom=10)
Map.centerObject(aoi_buffered)
# Add the 'double' band to the map
Map.addLayer(double_band, {'min': 0, 'max': 1, 'palette': ['blue', 'green', 'red']}, 'Double Band')

# Display the map
Map

In [ ]:
filtered_sentinel_data = get_harmonic_ts(year=year, aoi=aoi_buffered, time_intervals=time_intervals)

In [ ]:
filtered_sentinel_data

In [ ]:
Map = geemap.Map()


# Add the layer to the map.
ndvi_image = ee.Image(filtered_sentinel_data.get("fitted_data").toList(99).get(7))
Map.centerObject(aoi_buffered, 13)
ndvi_params = {'bands': ['fitted'], 'min': 0, 'max': 1, 'palette': ['white', 'green']}
Map.addLayer(ndvi_image, ndvi_params, 'NDVI FIRST', True)

# Display the map.
Map

# Testing Downscaling script

In [ ]:
from utils.downscale_LST_10m import generate_downscaled_lst

start_date = ee.Date("2018-01-01")
end_date = ee.Date("2018-12-31")
lst_band = "ST_B10"
landsat_bands_for_downscaling = [
    "SR_B2",
    "SR_B3",
    "SR_B4",
    "SR_B5",
    "SR_B6",
    "SR_B7",
    "NDVI",
    "NDWI",
    "NDBI",
]
sentinel_bands_for_downscaling = [
    "B2",
    "B3",
    "B4",
    "B8",
    "B11",
    "B12",
    "NDVI",
    "NDWI",
    "NDBI",
]
initial_scale = 30


year = 2019

# Define the time range for analysis
start_date = f"{year}-03-01"
end_date = f"{year}-10-31"

# Create time intervals
time_intervals = extract_time_ranges([start_date, end_date], 1)
filtered_sentinel_data = get_harmonic_ts(
    year=year, aoi=aoi_buffered, time_intervals=time_intervals
)

In [ ]:
# Get number of images in image collection
n_images = filtered_sentinel_data.get("fitted_data").size().getInfo()
print(f"Number of images in collection: {n_images}")

In [ ]:
Map = geemap.Map()


# Add the layer to the map.
ndvi_image = ee.Image(filtered_sentinel_data.get("fitted_data").toList(365).get(7))
Map.centerObject(aoi_buffered, 13)
ndvi_params = {'bands': ['fitted'], 'min': 0, 'max': 1, 'palette': ['white', 'green']}
Map.addLayer(ndvi_image, ndvi_params, 'NDVI FIRST', True)

# Display the map.
Map

In [ ]:
type(filtered_sentinel_data.get("fitted_data").first())

In [ ]:
# Select a valid Landsat 8 image with necessary bands
l8_image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(aoi_buffered)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt("CLOUD_COVER", 50))
    .first()
)



# Generate the downscaled LST
downscaled_lst = generate_downscaled_lst(
    lst_band,
    sentinel_bands_for_downscaling=sentinel_bands_for_downscaling,
    landsat_bands_for_downscaling=landsat_bands_for_downscaling,
    initial_scale=initial_scale,
    geometry=aoi_buffered,
    l8_image=l8_image,
    s2_image=filtered_sentinel_data.get("fitted_data").first(),
)

# Display the downscaled LST
Map = geemap.Map()
Map.centerObject(aoi_buffered, 10)
Map.addLayer(downscaled_lst, {"min": 0, "max": 40}, "Downscaled LST")

# Display the map
Map